In [8]:
from fastapi import FastAPI, Header, HTTPException
from pydantic import BaseModel
from typing import Optional, List
from rag.base import RetrievalChain 
from rag.json import JSONRetrievalChain
import requests
import uvicorn

app = FastAPI()
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=model, top_n=3)

class UserInfo(BaseModel):
    user_id: int
    name: str 
    age: int
    height: float
    weight: float
    workout_frequency: str
    workout_location: str
    gender: str
    job: str

class HabitInfo(BaseModel):
    category: str
    goal: str

class Action(BaseModel):
    action: str

class RoutineInfo(BaseModel):
    selected_actions: List[str]
    start_time: str
    repeat_days: str

class ActionResponse(BaseModel):
    actions: List[Action]

# RAG 초기화
rag = JSONRetrievalChain(source_uri=["pass"], docs=["pass"])
rag = rag.create_chain(cache_mode='load', local_db="./cached_healthcare/", category="의료", mode='kiwi', func="makeaction")
rag_retriever = rag.retriever
rag = rag.chain

@app.post("/make_action", response_model=dict)
async def make_action(
    user_info: UserInfo,
    habit_info: HabitInfo
):
    try:
        # create_chain의 키와 일치하도록 make_action_query 구성
        make_action_query = {
            "context": rag_retriever,
            "question": habit_info.goal,
            "user_id": user_info.user_id,
            "name": user_info.name, 
            "age": user_info.age,
            "height": user_info.height,
            "weight": user_info.weight,
            "workout_frequency": user_info.workout_frequency,
            "workout_location": user_info.workout_location,
            "gender": user_info.gender,
            "job": user_info.job,
            "category": habit_info.category,
            "goal": habit_info.goal
        }

        result = rag.invoke(make_action_query)
        
        # 결과를 actions_dict 형태로 변환
        actions_dict = {
            "actions": [result.actions[i].action for i in range(len(result.actions))]
        }
        
        # make_action_query와 actions_dict를 합쳐서 반환
        make_action_query.update(actions_dict)
        
        return make_action_query

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/make_routine", response_model=dict)
async def make_routine(
    make_action_result_query: dict,
    routine_info: RoutineInfo
):
    try:
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor, base_retriever=rag_retriever)
        # 압축된 문서를 사용하여 make_routine_query 구성
        compressed_docs = compression_retriever.invoke(make_action_result_query["goal"])
        
        make_routine_query = {
            "context": compressed_docs,
            "question": make_action_result_query["goal"],
            "user_id": make_action_result_query["user_id"],
            "name": make_action_result_query["name"],
            "age": make_action_result_query["age"],
            "gender": make_action_result_query["gender"],
            "job": make_action_result_query["job"],
            "weight": make_action_result_query["weight"],
            "height": make_action_result_query["height"],
            "workout_frequency": make_action_result_query["workout_frequency"],
            "workout_location": make_action_result_query["workout_location"],
            "category": make_action_result_query["category"],
            "goal": make_action_result_query["goal"],
            "selected_actions": routine_info.selected_actions,
            "start_time": routine_info.start_time,
            "repeat_days": routine_info.repeat_days
        }

        result = rag.invoke(make_routine_query)
        print(result)
        subroutine_dict = {
            "subroutines": [
                {
                    "emoji": sub_step.emoji,
                    "routine": sub_step.routine,
                    "secondDuration": sub_step.secondDuration
                } for sub_step in result.subRoutine  # result.subRoutine가 리스트라고 가정
            ]
        }
        return subroutine_dict
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/torch/distributions/constraint_registry.py:70: RuntimeWarning: coroutine 'Server.serve' was never awaited
  from torch.distributions import constraints, transforms
/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


로컬에서 벡터 저장소 로드 중
kiwi_bm25_의료 캐시 경로: cache/kiwi_bm25_의료
cache/kiwi_bm25_의료 로드 중
load_local 시간: 0.09초


RuntimeError: asyncio.run() cannot be called from a running event loop

### api_test

In [7]:

# API 테스트 코드
if __name__ == "__main__":
    # 테스트용 데이터
    test_user_info = {
        "user_id": 123,
        "name": "이상하",
        "age": 23,
        "height": 168,
        "weight": 65,
        "workout_frequency": "1회 미만",
        "workout_location": "집",
        "gender": "남성",
        "job": "학생"
    }

    test_habit_info = {
        "category": "의료",
        "goal": "탈모를 예방하고 진행 속도를 늦추고 싶어"
    }

    # API 엔드포인트 URL
    url = "http://localhost:8000/make_action"

    # POST 요청 보내기
    try:
        response = requests.post(
            url,
            json={
                "user_info": test_user_info,
                "habit_info": test_habit_info
            }
        )
        
        # 응답 확인
        if response.status_code == 200:
            print("Success!")
            print("Response:", response.json())
        else:
            print("Error:", response.status_code)
            print("Response:", response.text)
            
    except requests.exceptions.RequestException as e:
        print("Error making request:", e)

Success!
Response: {'actions': ['아침에 두피 마사지', '비타민D 보충제 섭취', '녹차 한 잔 마시기', '스트레스 관리 명상', '충분한 수분 섭취', '저녁에 두피 마사지', '올리브 오일 섭취', '생강차 마시기', '견과류 간식 먹기', '아침에 생선 섭취', '녹색 잎채소 먹기', '블루베리 간식 먹기', '저녁에 아보카도 먹기', '충분한 수면 취하기', '모발 건강 제품 사용', '주말에 레이저 치료', '모발 이식 상담', '항산화 식품 섭취', '스트레스 관리 운동', '주말에 전문가 상담']}
